# DRA算法+极小值攻击 

攻击方式：拜占庭用户在每次迭代中始终保持相同的恶意动态一致平均变量（不进行更新）

In [1]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [2]:
#定义变量 
agent_resource_quantity_attack1 = np.zeros(shape=(100,60000)) #100个agent的资源量 定义成一个矩阵 100*2000 
average_resource_honest_attack1=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户资源量的平均值
x_average_resource_quantity_attack1=np.zeros(shape=(100,60000)) #动态一致平均算法追踪平均值的变量
average_cost_honest_attack1=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户成本的平均值
iteration_update_attack1 = 0.6
dual_variable_attack1 = np.zeros(shape=(100,60000))#对偶变量
proportion_attack1 = 0.06 #100个用户中拜占庭用户的比例 100*proportion_attack1=拜占庭用户数量
W_attack1 =np.loadtxt("100_Network_Topology_Adjacency_Matrix_B_0.0625(the value of bii=0.0625).txt")
iteration_attack1=[]#绘图横坐标
a_attack1=np.loadtxt("100_uniform_distribution_(1,2).txt")#每个agent的成本函数中的超参数
b_attack1=np.loadtxt("100_normal_distribution_(2,0.5).txt")#每个agent的成本函数中的超参数

In [3]:
#用户对偶变量的初始化
for i in range (0,100-int(proportion_attack1*100)):
    dual_variable_attack1[i][0]=0
    
#用户资源量的初始化
for i in range (0,100-int(proportion_attack1*100)):
    agent_resource_quantity_attack1[i][0]=0
    
#诚实用户追踪均值变量的初始化 x(0)=agent_resource_quantity(0)
for i in range (0,100-int(proportion_attack1*100)):
    x_average_resource_quantity_attack1[i][0]=agent_resource_quantity_attack1[i][0]
    
#拜占庭用户追踪均值变量的初始化
for i in range (100-int((proportion_attack1*100)),100):
    x_average_resource_quantity_attack1[i][0]=0
    
#诚实用户的平均资源量初始化
sum_attack1=0
for i in range (0,100-int(proportion_attack1*100)):
    sum_attack1 = sum_attack1 + agent_resource_quantity_attack1[i][0]
average_resource_honest_attack1[0][0]=sum_attack1/(100-int(proportion_attack1*100))

#诚实用户平均成本初始化
sum_attack1=0
for i in range(0,100-int(proportion_attack1*100)):
    sum_attack1=sum_attack1+a_attack1[i]*(agent_resource_quantity_attack1[i][0]-b_attack1[i])*(agent_resource_quantity_attack1[i][0]-b_attack1[i])
average_cost_honest_attack1[0][0]=sum_attack1/(100-int(proportion_attack1*100))

iteration_attack1.append(0)

In [4]:
 def DRA_attack1(a_attack1,b_attack1,agent_resource_quantity_attack1,average_resource_honest_attack1,average_cost_honest_attack1,iteration_update_attack1,dual_variable_attack1,W_attack1,iteration_attack1,x_average_resource_quantity_attack1):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DRA算法+极小值拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_attack1[i][0])
    print("*************************")
    print("初始诚实用户资源申请量平均值avg为:",average_resource_honest_attack1[0][0])
    print("*************************")
    print("初始诚实用户平均成本为:",average_cost_honest_attack1[0][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_attack1[i][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始追踪平均量为:",x_average_resource_quantity_attack1[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_attack1)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,60000):#迭代次数
        iteration_attack1.append(k)       
        
        #诚实用户对偶变量的迭代
        for i in range(0,100-int(proportion_attack1*100)):
            dual_variable_attack1[i][k]=dual_variable_attack1[i][k-1]+iteration_update_attack1*((x_average_resource_quantity_attack1[i][k-1])-(50))
        
        #拜占庭用户对偶变量的迭代
        for i in range (100-int((proportion_attack1*100)),100):
            dual_variable_attack1[i][k]=dual_variable_attack1[i][k-1]
        
        #诚实用户资源申请量的迭代
        for i in range (0,100-int(proportion_attack1*100)):
            agent_resource_quantity_attack1[i][k]=agent_resource_quantity_attack1[i][k-1]-iteration_update_attack1*(((2*a_attack1[i]*agent_resource_quantity_attack1[i][k-1]-2*a_attack1[i]*b_attack1[i])/100)+(dual_variable_attack1[i][k]/100))
            #原变量的投影操作
            if agent_resource_quantity_attack1[i][k] < 0:
                agent_resource_quantity_attack1[i][k]=0
            elif agent_resource_quantity_attack1[i][k] > 100: 
                agent_resource_quantity_attack1[i][k]=100
                
        #拜占庭用户资源申请量的迭代
        for i in range (100-int((proportion_attack1*100)),100):
            agent_resource_quantity_attack1[i][k]=agent_resource_quantity_attack1[i][k-1]
                
        #诚实用户平均值追踪量的迭代 最新版 Wx+资源量差值
        for i in range (0,100-int(proportion_attack1*100)):
            sum_x_attack1=0
            for j in range (0,100):
                if W_attack1[i][j]!=0:
                    sum_x_attack1=sum_x_attack1+W_attack1[i][j]*x_average_resource_quantity_attack1[j][k-1]
            x_average_resource_quantity_attack1[i][k]=sum_x_attack1+(agent_resource_quantity_attack1[i][k]-agent_resource_quantity_attack1[i][k-1])        
        
        #拜占庭用户平均值追踪量的迭代 最新版 Wx+资源量差值
        for i in range (100-int((proportion_attack1*100)),100):
            x_average_resource_quantity_attack1[i][k]=x_average_resource_quantity_attack1[i][k-1]     
        
        #计算所有诚实用户的平均资源量
        sum1_attack1=0
        for i in range (0,100-int(proportion_attack1*100)):
            sum1_attack1 = sum1_attack1 + agent_resource_quantity_attack1[i][k]
        average_resource_honest_attack1[0][k]=sum1_attack1/(100-int(proportion_attack1*100))
        
        #计算所有诚实用户的平均成本
        sum2_attack1=0
        for i in range(0,100-int(proportion_attack1*100)):
            sum2_attack1=sum2_attack1+a_attack1[i]*(agent_resource_quantity_attack1[i][k]-b_attack1[i])*(agent_resource_quantity_attack1[i][k]-b_attack1[i])
        average_cost_honest_attack1[0][k]=sum2_attack1/(100-int(proportion_attack1*100))
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_attack1[i][k])
            print("资源申请量平均值avg为:",average_resource_honest_attack1[0][k])
            print("****************************************************************************")
            print("平均成本为:",average_cost_honest_attack1[0][k])
            print("****************************************************************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_attack1[i][k]) 
            print("****************************************************************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的的追踪平均量为:",x_average_resource_quantity_attack1[i][k])
            print("*************************")
            print("迭代步长为:",iteration_update_attack1)
            print("****************************************************************************")
        
        #iteration_update_attack1=1/(pow(k+1,0.5)) #递减步长
    
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [5]:
DRA_attack1(a_attack1,b_attack1,agent_resource_quantity_attack1,average_resource_honest_attack1,average_cost_honest_attack1,iteration_update_attack1,dual_variable_attack1,W_attack1,iteration_attack1,x_average_resource_quantity_attack1)

****************************DRA算法+极小值拜占庭攻击****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 0.0
agent 2 的初始资源申请量为: 0.0
agent 3 的初始资源申请量为: 0.0
agent 4 的初始资源申请量为: 0.0
agent 5 的初始资源申请量为: 0.0
agent 6 的初始资源申请量为: 0.0
agent 7 的初始资源申请量为: 0.0
agent 8 的初始资源申请量为: 0.0
agent 9 的初始资源申请量为: 0.0
agent 10 的初始资源申请量为: 0.0
agent 11 的初始资源申请量为: 0.0
agent 12 的初始资源申请量为: 0.0
agent 13 的初始资源申请量为: 0.0
agent 14 的初始资源申请量为: 0.0
agent 15 的初始资源申请量为: 0.0
agent 16 的初始资源申请量为: 0.0
agent 17 的初始资源申请量为: 0.0
agent 18 的初始资源申请量为: 0.0
agent 19 的初始资源申请量为: 0.0
agent 20 的初始资源申请量为: 0.0
agent 21 的初始资源申请量为: 0.0
agent 22 的初始资源申请量为: 0.0
agent 23 的初始资源申请量为: 0.0
agent 24 的初始资源申请量为: 0.0
agent 25 的初始资源申请量为: 0.0
agent 26 的初始资源申请量为: 0.0
agent 27 的初始资源申请量为: 0.0
agent 28 的初始资源申请量为: 0.0
agent 29 的初始资源申请量为: 0.0
agent 30 的初始资源申请量为: 0.0
agent 31 的初始资源申请量为: 0.0
agent 32 的初始资源申请量为: 0.0
agent 33 的初始资源申请量为: 0.0
agent 34 的初始资源申请量为: 0.0
agent 35 的初始资源申请量为: 0.0
agent 36 的初始资源申请量为: 0.0
agent 37 的初始资源申请量为: 0.0
agent 38 的初始资源申请量为: 0.0
agent 39 的初始资源申请

3999 次迭代中的结果：
agent 1 在 3999 次迭代中的资源申请量为: 100.0
agent 2 在 3999 次迭代中的资源申请量为: 100.0
agent 3 在 3999 次迭代中的资源申请量为: 100.0
agent 4 在 3999 次迭代中的资源申请量为: 100.0
agent 5 在 3999 次迭代中的资源申请量为: 100.0
agent 6 在 3999 次迭代中的资源申请量为: 100.0
agent 7 在 3999 次迭代中的资源申请量为: 100.0
agent 8 在 3999 次迭代中的资源申请量为: 100.0
agent 9 在 3999 次迭代中的资源申请量为: 100.0
agent 10 在 3999 次迭代中的资源申请量为: 100.0
agent 11 在 3999 次迭代中的资源申请量为: 100.0
agent 12 在 3999 次迭代中的资源申请量为: 100.0
agent 13 在 3999 次迭代中的资源申请量为: 100.0
agent 14 在 3999 次迭代中的资源申请量为: 100.0
agent 15 在 3999 次迭代中的资源申请量为: 100.0
agent 16 在 3999 次迭代中的资源申请量为: 100.0
agent 17 在 3999 次迭代中的资源申请量为: 100.0
agent 18 在 3999 次迭代中的资源申请量为: 100.0
agent 19 在 3999 次迭代中的资源申请量为: 100.0
agent 20 在 3999 次迭代中的资源申请量为: 100.0
agent 21 在 3999 次迭代中的资源申请量为: 100.0
agent 22 在 3999 次迭代中的资源申请量为: 100.0
agent 23 在 3999 次迭代中的资源申请量为: 100.0
agent 24 在 3999 次迭代中的资源申请量为: 100.0
agent 25 在 3999 次迭代中的资源申请量为: 100.0
agent 26 在 3999 次迭代中的资源申请量为: 100.0
agent 27 在 3999 次迭代中的资源申请量为: 100.0
agent 28 在 3999 次迭代中的资源申请量为: 100.0
agent 29 在 3999

5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 100.0
agent 2 在 5999 次迭代中的资源申请量为: 100.0
agent 3 在 5999 次迭代中的资源申请量为: 100.0
agent 4 在 5999 次迭代中的资源申请量为: 100.0
agent 5 在 5999 次迭代中的资源申请量为: 100.0
agent 6 在 5999 次迭代中的资源申请量为: 100.0
agent 7 在 5999 次迭代中的资源申请量为: 100.0
agent 8 在 5999 次迭代中的资源申请量为: 100.0
agent 9 在 5999 次迭代中的资源申请量为: 100.0
agent 10 在 5999 次迭代中的资源申请量为: 100.0
agent 11 在 5999 次迭代中的资源申请量为: 100.0
agent 12 在 5999 次迭代中的资源申请量为: 100.0
agent 13 在 5999 次迭代中的资源申请量为: 100.0
agent 14 在 5999 次迭代中的资源申请量为: 100.0
agent 15 在 5999 次迭代中的资源申请量为: 100.0
agent 16 在 5999 次迭代中的资源申请量为: 100.0
agent 17 在 5999 次迭代中的资源申请量为: 100.0
agent 18 在 5999 次迭代中的资源申请量为: 100.0
agent 19 在 5999 次迭代中的资源申请量为: 100.0
agent 20 在 5999 次迭代中的资源申请量为: 100.0
agent 21 在 5999 次迭代中的资源申请量为: 100.0
agent 22 在 5999 次迭代中的资源申请量为: 100.0
agent 23 在 5999 次迭代中的资源申请量为: 100.0
agent 24 在 5999 次迭代中的资源申请量为: 100.0
agent 25 在 5999 次迭代中的资源申请量为: 100.0
agent 26 在 5999 次迭代中的资源申请量为: 100.0
agent 27 在 5999 次迭代中的资源申请量为: 100.0
agent 28 在 5999 次迭代中的资源申请量为: 100.0
agent 29 在 5999

7999 次迭代中的结果：
agent 1 在 7999 次迭代中的资源申请量为: 100.0
agent 2 在 7999 次迭代中的资源申请量为: 100.0
agent 3 在 7999 次迭代中的资源申请量为: 100.0
agent 4 在 7999 次迭代中的资源申请量为: 100.0
agent 5 在 7999 次迭代中的资源申请量为: 100.0
agent 6 在 7999 次迭代中的资源申请量为: 100.0
agent 7 在 7999 次迭代中的资源申请量为: 100.0
agent 8 在 7999 次迭代中的资源申请量为: 100.0
agent 9 在 7999 次迭代中的资源申请量为: 100.0
agent 10 在 7999 次迭代中的资源申请量为: 100.0
agent 11 在 7999 次迭代中的资源申请量为: 100.0
agent 12 在 7999 次迭代中的资源申请量为: 100.0
agent 13 在 7999 次迭代中的资源申请量为: 100.0
agent 14 在 7999 次迭代中的资源申请量为: 100.0
agent 15 在 7999 次迭代中的资源申请量为: 100.0
agent 16 在 7999 次迭代中的资源申请量为: 100.0
agent 17 在 7999 次迭代中的资源申请量为: 100.0
agent 18 在 7999 次迭代中的资源申请量为: 100.0
agent 19 在 7999 次迭代中的资源申请量为: 100.0
agent 20 在 7999 次迭代中的资源申请量为: 100.0
agent 21 在 7999 次迭代中的资源申请量为: 100.0
agent 22 在 7999 次迭代中的资源申请量为: 100.0
agent 23 在 7999 次迭代中的资源申请量为: 100.0
agent 24 在 7999 次迭代中的资源申请量为: 100.0
agent 25 在 7999 次迭代中的资源申请量为: 100.0
agent 26 在 7999 次迭代中的资源申请量为: 100.0
agent 27 在 7999 次迭代中的资源申请量为: 100.0
agent 28 在 7999 次迭代中的资源申请量为: 100.0
agent 29 在 7999

9999 次迭代中的结果：
agent 1 在 9999 次迭代中的资源申请量为: 100.0
agent 2 在 9999 次迭代中的资源申请量为: 100.0
agent 3 在 9999 次迭代中的资源申请量为: 100.0
agent 4 在 9999 次迭代中的资源申请量为: 100.0
agent 5 在 9999 次迭代中的资源申请量为: 100.0
agent 6 在 9999 次迭代中的资源申请量为: 100.0
agent 7 在 9999 次迭代中的资源申请量为: 100.0
agent 8 在 9999 次迭代中的资源申请量为: 100.0
agent 9 在 9999 次迭代中的资源申请量为: 100.0
agent 10 在 9999 次迭代中的资源申请量为: 100.0
agent 11 在 9999 次迭代中的资源申请量为: 100.0
agent 12 在 9999 次迭代中的资源申请量为: 100.0
agent 13 在 9999 次迭代中的资源申请量为: 100.0
agent 14 在 9999 次迭代中的资源申请量为: 100.0
agent 15 在 9999 次迭代中的资源申请量为: 100.0
agent 16 在 9999 次迭代中的资源申请量为: 100.0
agent 17 在 9999 次迭代中的资源申请量为: 100.0
agent 18 在 9999 次迭代中的资源申请量为: 100.0
agent 19 在 9999 次迭代中的资源申请量为: 100.0
agent 20 在 9999 次迭代中的资源申请量为: 100.0
agent 21 在 9999 次迭代中的资源申请量为: 100.0
agent 22 在 9999 次迭代中的资源申请量为: 100.0
agent 23 在 9999 次迭代中的资源申请量为: 100.0
agent 24 在 9999 次迭代中的资源申请量为: 100.0
agent 25 在 9999 次迭代中的资源申请量为: 100.0
agent 26 在 9999 次迭代中的资源申请量为: 100.0
agent 27 在 9999 次迭代中的资源申请量为: 100.0
agent 28 在 9999 次迭代中的资源申请量为: 100.0
agent 29 在 9999

...

51999 次迭代中的结果：
agent 1 在 51999 次迭代中的资源申请量为: 100.0
agent 2 在 51999 次迭代中的资源申请量为: 100.0
agent 3 在 51999 次迭代中的资源申请量为: 100.0
agent 4 在 51999 次迭代中的资源申请量为: 100.0
agent 5 在 51999 次迭代中的资源申请量为: 100.0
agent 6 在 51999 次迭代中的资源申请量为: 100.0
agent 7 在 51999 次迭代中的资源申请量为: 100.0
agent 8 在 51999 次迭代中的资源申请量为: 100.0
agent 9 在 51999 次迭代中的资源申请量为: 100.0
agent 10 在 51999 次迭代中的资源申请量为: 100.0
agent 11 在 51999 次迭代中的资源申请量为: 100.0
agent 12 在 51999 次迭代中的资源申请量为: 100.0
agent 13 在 51999 次迭代中的资源申请量为: 100.0
agent 14 在 51999 次迭代中的资源申请量为: 100.0
agent 15 在 51999 次迭代中的资源申请量为: 100.0
agent 16 在 51999 次迭代中的资源申请量为: 100.0
agent 17 在 51999 次迭代中的资源申请量为: 100.0
agent 18 在 51999 次迭代中的资源申请量为: 100.0
agent 19 在 51999 次迭代中的资源申请量为: 100.0
agent 20 在 51999 次迭代中的资源申请量为: 100.0
agent 21 在 51999 次迭代中的资源申请量为: 100.0
agent 22 在 51999 次迭代中的资源申请量为: 100.0
agent 23 在 51999 次迭代中的资源申请量为: 100.0
agent 24 在 51999 次迭代中的资源申请量为: 100.0
agent 25 在 51999 次迭代中的资源申请量为: 100.0
agent 26 在 51999 次迭代中的资源申请量为: 100.0
agent 27 在 51999 次迭代中的资源申请量为: 100.0
agent 28 在 51999 次迭代中的

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 100.0
agent 2 在 53999 次迭代中的资源申请量为: 100.0
agent 3 在 53999 次迭代中的资源申请量为: 100.0
agent 4 在 53999 次迭代中的资源申请量为: 100.0
agent 5 在 53999 次迭代中的资源申请量为: 100.0
agent 6 在 53999 次迭代中的资源申请量为: 100.0
agent 7 在 53999 次迭代中的资源申请量为: 100.0
agent 8 在 53999 次迭代中的资源申请量为: 100.0
agent 9 在 53999 次迭代中的资源申请量为: 100.0
agent 10 在 53999 次迭代中的资源申请量为: 100.0
agent 11 在 53999 次迭代中的资源申请量为: 100.0
agent 12 在 53999 次迭代中的资源申请量为: 100.0
agent 13 在 53999 次迭代中的资源申请量为: 100.0
agent 14 在 53999 次迭代中的资源申请量为: 100.0
agent 15 在 53999 次迭代中的资源申请量为: 100.0
agent 16 在 53999 次迭代中的资源申请量为: 100.0
agent 17 在 53999 次迭代中的资源申请量为: 100.0
agent 18 在 53999 次迭代中的资源申请量为: 100.0
agent 19 在 53999 次迭代中的资源申请量为: 100.0
agent 20 在 53999 次迭代中的资源申请量为: 100.0
agent 21 在 53999 次迭代中的资源申请量为: 100.0
agent 22 在 53999 次迭代中的资源申请量为: 100.0
agent 23 在 53999 次迭代中的资源申请量为: 100.0
agent 24 在 53999 次迭代中的资源申请量为: 100.0
agent 25 在 53999 次迭代中的资源申请量为: 100.0
agent 26 在 53999 次迭代中的资源申请量为: 100.0
agent 27 在 53999 次迭代中的资源申请量为: 100.0
agent 28 在 53999 次迭代中的

55999 次迭代中的结果：
agent 1 在 55999 次迭代中的资源申请量为: 100.0
agent 2 在 55999 次迭代中的资源申请量为: 100.0
agent 3 在 55999 次迭代中的资源申请量为: 100.0
agent 4 在 55999 次迭代中的资源申请量为: 100.0
agent 5 在 55999 次迭代中的资源申请量为: 100.0
agent 6 在 55999 次迭代中的资源申请量为: 100.0
agent 7 在 55999 次迭代中的资源申请量为: 100.0
agent 8 在 55999 次迭代中的资源申请量为: 100.0
agent 9 在 55999 次迭代中的资源申请量为: 100.0
agent 10 在 55999 次迭代中的资源申请量为: 100.0
agent 11 在 55999 次迭代中的资源申请量为: 100.0
agent 12 在 55999 次迭代中的资源申请量为: 100.0
agent 13 在 55999 次迭代中的资源申请量为: 100.0
agent 14 在 55999 次迭代中的资源申请量为: 100.0
agent 15 在 55999 次迭代中的资源申请量为: 100.0
agent 16 在 55999 次迭代中的资源申请量为: 100.0
agent 17 在 55999 次迭代中的资源申请量为: 100.0
agent 18 在 55999 次迭代中的资源申请量为: 100.0
agent 19 在 55999 次迭代中的资源申请量为: 100.0
agent 20 在 55999 次迭代中的资源申请量为: 100.0
agent 21 在 55999 次迭代中的资源申请量为: 100.0
agent 22 在 55999 次迭代中的资源申请量为: 100.0
agent 23 在 55999 次迭代中的资源申请量为: 100.0
agent 24 在 55999 次迭代中的资源申请量为: 100.0
agent 25 在 55999 次迭代中的资源申请量为: 100.0
agent 26 在 55999 次迭代中的资源申请量为: 100.0
agent 27 在 55999 次迭代中的资源申请量为: 100.0
agent 28 在 55999 次迭代中的

57999 次迭代中的结果：
agent 1 在 57999 次迭代中的资源申请量为: 100.0
agent 2 在 57999 次迭代中的资源申请量为: 100.0
agent 3 在 57999 次迭代中的资源申请量为: 100.0
agent 4 在 57999 次迭代中的资源申请量为: 100.0
agent 5 在 57999 次迭代中的资源申请量为: 100.0
agent 6 在 57999 次迭代中的资源申请量为: 100.0
agent 7 在 57999 次迭代中的资源申请量为: 100.0
agent 8 在 57999 次迭代中的资源申请量为: 100.0
agent 9 在 57999 次迭代中的资源申请量为: 100.0
agent 10 在 57999 次迭代中的资源申请量为: 100.0
agent 11 在 57999 次迭代中的资源申请量为: 100.0
agent 12 在 57999 次迭代中的资源申请量为: 100.0
agent 13 在 57999 次迭代中的资源申请量为: 100.0
agent 14 在 57999 次迭代中的资源申请量为: 100.0
agent 15 在 57999 次迭代中的资源申请量为: 100.0
agent 16 在 57999 次迭代中的资源申请量为: 100.0
agent 17 在 57999 次迭代中的资源申请量为: 100.0
agent 18 在 57999 次迭代中的资源申请量为: 100.0
agent 19 在 57999 次迭代中的资源申请量为: 100.0
agent 20 在 57999 次迭代中的资源申请量为: 100.0
agent 21 在 57999 次迭代中的资源申请量为: 100.0
agent 22 在 57999 次迭代中的资源申请量为: 100.0
agent 23 在 57999 次迭代中的资源申请量为: 100.0
agent 24 在 57999 次迭代中的资源申请量为: 100.0
agent 25 在 57999 次迭代中的资源申请量为: 100.0
agent 26 在 57999 次迭代中的资源申请量为: 100.0
agent 27 在 57999 次迭代中的资源申请量为: 100.0
agent 28 在 57999 次迭代中的

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 100.0
agent 2 在 59999 次迭代中的资源申请量为: 100.0
agent 3 在 59999 次迭代中的资源申请量为: 100.0
agent 4 在 59999 次迭代中的资源申请量为: 100.0
agent 5 在 59999 次迭代中的资源申请量为: 100.0
agent 6 在 59999 次迭代中的资源申请量为: 100.0
agent 7 在 59999 次迭代中的资源申请量为: 100.0
agent 8 在 59999 次迭代中的资源申请量为: 100.0
agent 9 在 59999 次迭代中的资源申请量为: 100.0
agent 10 在 59999 次迭代中的资源申请量为: 100.0
agent 11 在 59999 次迭代中的资源申请量为: 100.0
agent 12 在 59999 次迭代中的资源申请量为: 100.0
agent 13 在 59999 次迭代中的资源申请量为: 100.0
agent 14 在 59999 次迭代中的资源申请量为: 100.0
agent 15 在 59999 次迭代中的资源申请量为: 100.0
agent 16 在 59999 次迭代中的资源申请量为: 100.0
agent 17 在 59999 次迭代中的资源申请量为: 100.0
agent 18 在 59999 次迭代中的资源申请量为: 100.0
agent 19 在 59999 次迭代中的资源申请量为: 100.0
agent 20 在 59999 次迭代中的资源申请量为: 100.0
agent 21 在 59999 次迭代中的资源申请量为: 100.0
agent 22 在 59999 次迭代中的资源申请量为: 100.0
agent 23 在 59999 次迭代中的资源申请量为: 100.0
agent 24 在 59999 次迭代中的资源申请量为: 100.0
agent 25 在 59999 次迭代中的资源申请量为: 100.0
agent 26 在 59999 次迭代中的资源申请量为: 100.0
agent 27 在 59999 次迭代中的资源申请量为: 100.0
agent 28 在 59999 次迭代中的

In [6]:
np.savetxt('./data/00_DRA_agent_resource_quantity_min_attack.txt',agent_resource_quantity_attack1,fmt='%0.8f')#用户资源
np.savetxt('./data/00_DRA_average_cost_honest_min_attack.txt',average_cost_honest_attack1,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/00_DRA_average_resource_honest_min_attack.txt',average_resource_honest_attack1,fmt='%0.8f')#诚实用户平均资源
np.savetxt('./data/00_DRA_dual_variable_min_attack.txt',dual_variable_attack1,fmt='%0.8f')#用户对偶变量

# DRA算法+极大值攻击

In [6]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [7]:
#定义变量 
agent_resource_quantity_attack2 = np.zeros(shape=(100,60000)) #100个agent的资源量 定义成一个矩阵 100*2000 
average_resource_honest_attack2=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户资源量的平均值
x_average_resource_quantity_attack2=np.zeros(shape=(100,60000)) #动态一致平均算法追踪平均值的变量
average_cost_honest_attack2=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户成本的平均值
iteration_update_attack2 = 0.6
dual_variable_attack2 = np.zeros(shape=(100,60000))#对偶变量
proportion_attack2 = 0.06 #100个用户中拜占庭用户的比例 100*proportion_attack2=拜占庭用户数量
W_attack2 =np.loadtxt("100_Network_Topology_Adjacency_Matrix_B_0.0625(the value of bii=0.0625).txt")
iteration_attack2=[]#绘图横坐标
a_attack2=np.loadtxt("100_uniform_distribution_(1,2).txt")#每个agent的成本函数中的超参数
b_attack2=np.loadtxt("100_normal_distribution_(2,0.5).txt")#每个agent的成本函数中的超参数

In [8]:
#诚实用户对偶变量的初始化
for i in range (0,100-int(proportion_attack2*100)):
    dual_variable_attack2[i][0]=0

#诚实用户资源量的初始化
for i in range (0,100-int(proportion_attack2*100)):
    agent_resource_quantity_attack2[i][0]=0
    
#诚实用户追踪均值变量的初始化 x(0)=agent_resource_quantity(0)
for i in range (0,100-int(proportion_attack2*100)):
    x_average_resource_quantity_attack2[i][0]=agent_resource_quantity_attack2[i][0]
    
#拜占庭用户追踪均值变量的初始化
for i in range (100-int((proportion_attack2*100)),100):
    x_average_resource_quantity_attack2[i][0]=100
    
#诚实用户的平均资源量初始化
sum_attack2=0
for i in range (0,100-int(proportion_attack2*100)):
    sum_attack2 = sum_attack2 + agent_resource_quantity_attack2[i][0]
average_resource_honest_attack2[0][0]=sum_attack2/(100-int(proportion_attack2*100))

#诚实用户平均成本初始化
sum_attack2=0
for i in range(0,100-int(proportion_attack2*100)):
    sum_attack2=sum_attack2+a_attack2[i]*(agent_resource_quantity_attack2[i][0]-b_attack2[i])*(agent_resource_quantity_attack2[i][0]-b_attack2[i])
average_cost_honest_attack2[0][0]=sum_attack2/(100-int(proportion_attack2*100))

iteration_attack2.append(0)

In [9]:
def DRA_attack2(a_attack2,b_attack2,agent_resource_quantity_attack2,average_resource_honest_attack2,average_cost_honest_attack2,iteration_update_attack2,dual_variable_attack2,W_attack2,iteration_attack2,x_average_resource_quantity_attack2):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DRA算法+极大值拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_attack2[i][0])
    print("*************************")
    print("初始诚实用户资源申请量平均值avg为:",average_resource_honest_attack2[0][0])
    print("*************************")
    print("初始诚实用户平均成本为:",average_cost_honest_attack2[0][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_attack2[i][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始追踪平均量为:",x_average_resource_quantity_attack2[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_attack2)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,60000):#迭代次数
        iteration_attack2.append(k)

        #诚实用户对偶变量的迭代
        for i in range(0,100-int(proportion_attack2*100)):
            dual_variable_attack2[i][k]=dual_variable_attack2[i][k-1]+iteration_update_attack2*((x_average_resource_quantity_attack2[i][k-1])-(50))
        
        #拜占庭用户对偶变量的迭代
        for i in range (100-int((proportion_attack2*100)),100):
            dual_variable_attack2[i][k]=dual_variable_attack2[i][k-1]
        
        #诚实用户资源申请量的迭代
        for i in range (0,100-int(proportion_attack2*100)):
            agent_resource_quantity_attack2[i][k]=agent_resource_quantity_attack2[i][k-1]-iteration_update_attack2*(((2*a_attack2[i]*agent_resource_quantity_attack2[i][k-1]-2*a_attack2[i]*b_attack2[i])/100)+(dual_variable_attack2[i][k]/100))
            #原变量的投影操作
            if agent_resource_quantity_attack2[i][k] < 0:
                agent_resource_quantity_attack2[i][k]=0
            elif agent_resource_quantity_attack2[i][k] > 100: 
                agent_resource_quantity_attack2[i][k]=100
                
        #拜占庭用户资源申请量的迭代
        for i in range (100-int((proportion_attack2*100)),100):
            agent_resource_quantity_attack2[i][k]=agent_resource_quantity_attack2[i][k-1]
                
        #诚实用户平均值追踪量的迭代 最新版 Wx+资源量差值
        for i in range (0,100-int(proportion_attack2*100)):
            sum_x_attack2=0
            for j in range (0,100):
                if W_attack2[i][j]!=0:
                    sum_x_attack2=sum_x_attack2+W_attack2[i][j]*x_average_resource_quantity_attack2[j][k-1]
            x_average_resource_quantity_attack2[i][k]=sum_x_attack2+(agent_resource_quantity_attack2[i][k]-agent_resource_quantity_attack2[i][k-1])        
        
        #拜占庭用户平均值追踪量的迭代 最新版 Wx+资源量差值
        for i in range (100-int((proportion_attack2*100)),100):
            x_average_resource_quantity_attack2[i][k]=x_average_resource_quantity_attack2[i][k-1]         
        
        #计算所有诚实用户的平均资源量
        sum1_attack2=0
        for i in range (0,100-int(proportion_attack2*100)):
            sum1_attack2 = sum1_attack2 + agent_resource_quantity_attack2[i][k]
        average_resource_honest_attack2[0][k]=sum1_attack2/(100-int(proportion_attack2*100))
        
        #计算所有诚实用户的平均成本
        sum2_attack2=0
        for i in range(0,100-int(proportion_attack2*100)):
            sum2_attack2=sum2_attack2+a_attack2[i]*(agent_resource_quantity_attack2[i][k]-b_attack2[i])*(agent_resource_quantity_attack2[i][k]-b_attack2[i])
        average_cost_honest_attack2[0][k]=sum2_attack2/(100-int(proportion_attack2*100))
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_attack2[i][k])
            print("资源申请量平均值avg为:",average_resource_honest_attack2[0][k])
            print("****************************************************************************")
            print("平均成本为:",average_cost_honest_attack2[0][k])
            print("****************************************************************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_attack2[i][k])   
            print("*************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的的追踪平均量为:",x_average_resource_quantity_attack2[i][k])
            print("*************************")
            print("迭代步长为:",iteration_update_attack2)
            print("****************************************************************************")
        
        #iteration_update_attack2=1/(pow(k+1,0.5)) #递减步长
    
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [10]:
DRA_attack2(a_attack2,b_attack2,agent_resource_quantity_attack2,average_resource_honest_attack2,average_cost_honest_attack2,iteration_update_attack2,dual_variable_attack2,W_attack2,iteration_attack2,x_average_resource_quantity_attack2)

****************************DRA算法+极大值拜占庭攻击****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 0.0
agent 2 的初始资源申请量为: 0.0
agent 3 的初始资源申请量为: 0.0
agent 4 的初始资源申请量为: 0.0
agent 5 的初始资源申请量为: 0.0
agent 6 的初始资源申请量为: 0.0
agent 7 的初始资源申请量为: 0.0
agent 8 的初始资源申请量为: 0.0
agent 9 的初始资源申请量为: 0.0
agent 10 的初始资源申请量为: 0.0
agent 11 的初始资源申请量为: 0.0
agent 12 的初始资源申请量为: 0.0
agent 13 的初始资源申请量为: 0.0
agent 14 的初始资源申请量为: 0.0
agent 15 的初始资源申请量为: 0.0
agent 16 的初始资源申请量为: 0.0
agent 17 的初始资源申请量为: 0.0
agent 18 的初始资源申请量为: 0.0
agent 19 的初始资源申请量为: 0.0
agent 20 的初始资源申请量为: 0.0
agent 21 的初始资源申请量为: 0.0
agent 22 的初始资源申请量为: 0.0
agent 23 的初始资源申请量为: 0.0
agent 24 的初始资源申请量为: 0.0
agent 25 的初始资源申请量为: 0.0
agent 26 的初始资源申请量为: 0.0
agent 27 的初始资源申请量为: 0.0
agent 28 的初始资源申请量为: 0.0
agent 29 的初始资源申请量为: 0.0
agent 30 的初始资源申请量为: 0.0
agent 31 的初始资源申请量为: 0.0
agent 32 的初始资源申请量为: 0.0
agent 33 的初始资源申请量为: 0.0
agent 34 的初始资源申请量为: 0.0
agent 35 的初始资源申请量为: 0.0
agent 36 的初始资源申请量为: 0.0
agent 37 的初始资源申请量为: 0.0
agent 38 的初始资源申请量为: 0.0
agent 39 的初始资源申请

3999 次迭代中的结果：
agent 1 在 3999 次迭代中的资源申请量为: 0.0
agent 2 在 3999 次迭代中的资源申请量为: 0.0
agent 3 在 3999 次迭代中的资源申请量为: 0.0
agent 4 在 3999 次迭代中的资源申请量为: 0.0
agent 5 在 3999 次迭代中的资源申请量为: 0.0
agent 6 在 3999 次迭代中的资源申请量为: 0.0
agent 7 在 3999 次迭代中的资源申请量为: 0.0
agent 8 在 3999 次迭代中的资源申请量为: 0.0
agent 9 在 3999 次迭代中的资源申请量为: 0.0
agent 10 在 3999 次迭代中的资源申请量为: 0.0
agent 11 在 3999 次迭代中的资源申请量为: 0.0
agent 12 在 3999 次迭代中的资源申请量为: 0.0
agent 13 在 3999 次迭代中的资源申请量为: 0.0
agent 14 在 3999 次迭代中的资源申请量为: 0.0
agent 15 在 3999 次迭代中的资源申请量为: 0.0
agent 16 在 3999 次迭代中的资源申请量为: 0.0
agent 17 在 3999 次迭代中的资源申请量为: 0.0
agent 18 在 3999 次迭代中的资源申请量为: 0.0
agent 19 在 3999 次迭代中的资源申请量为: 0.0
agent 20 在 3999 次迭代中的资源申请量为: 0.0
agent 21 在 3999 次迭代中的资源申请量为: 0.0
agent 22 在 3999 次迭代中的资源申请量为: 0.0
agent 23 在 3999 次迭代中的资源申请量为: 0.0
agent 24 在 3999 次迭代中的资源申请量为: 0.0
agent 25 在 3999 次迭代中的资源申请量为: 0.0
agent 26 在 3999 次迭代中的资源申请量为: 0.0
agent 27 在 3999 次迭代中的资源申请量为: 0.0
agent 28 在 3999 次迭代中的资源申请量为: 0.0
agent 29 在 3999 次迭代中的资源申请量为: 0.0
agent 30 在 3999 次迭代中的资源申请量为: 0.0
agent

5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 0.0
agent 2 在 5999 次迭代中的资源申请量为: 0.0
agent 3 在 5999 次迭代中的资源申请量为: 0.0
agent 4 在 5999 次迭代中的资源申请量为: 0.0
agent 5 在 5999 次迭代中的资源申请量为: 0.0
agent 6 在 5999 次迭代中的资源申请量为: 0.0
agent 7 在 5999 次迭代中的资源申请量为: 0.0
agent 8 在 5999 次迭代中的资源申请量为: 0.0
agent 9 在 5999 次迭代中的资源申请量为: 0.0
agent 10 在 5999 次迭代中的资源申请量为: 0.0
agent 11 在 5999 次迭代中的资源申请量为: 0.0
agent 12 在 5999 次迭代中的资源申请量为: 0.0
agent 13 在 5999 次迭代中的资源申请量为: 0.0
agent 14 在 5999 次迭代中的资源申请量为: 0.0
agent 15 在 5999 次迭代中的资源申请量为: 0.0
agent 16 在 5999 次迭代中的资源申请量为: 0.0
agent 17 在 5999 次迭代中的资源申请量为: 0.0
agent 18 在 5999 次迭代中的资源申请量为: 0.0
agent 19 在 5999 次迭代中的资源申请量为: 0.0
agent 20 在 5999 次迭代中的资源申请量为: 0.0
agent 21 在 5999 次迭代中的资源申请量为: 0.0
agent 22 在 5999 次迭代中的资源申请量为: 0.0
agent 23 在 5999 次迭代中的资源申请量为: 0.0
agent 24 在 5999 次迭代中的资源申请量为: 0.0
agent 25 在 5999 次迭代中的资源申请量为: 0.0
agent 26 在 5999 次迭代中的资源申请量为: 0.0
agent 27 在 5999 次迭代中的资源申请量为: 0.0
agent 28 在 5999 次迭代中的资源申请量为: 0.0
agent 29 在 5999 次迭代中的资源申请量为: 0.0
agent 30 在 5999 次迭代中的资源申请量为: 0.0
agent

7999 次迭代中的结果：
agent 1 在 7999 次迭代中的资源申请量为: 0.0
agent 2 在 7999 次迭代中的资源申请量为: 0.0
agent 3 在 7999 次迭代中的资源申请量为: 0.0
agent 4 在 7999 次迭代中的资源申请量为: 0.0
agent 5 在 7999 次迭代中的资源申请量为: 0.0
agent 6 在 7999 次迭代中的资源申请量为: 0.0
agent 7 在 7999 次迭代中的资源申请量为: 0.0
agent 8 在 7999 次迭代中的资源申请量为: 0.0
agent 9 在 7999 次迭代中的资源申请量为: 0.0
agent 10 在 7999 次迭代中的资源申请量为: 0.0
agent 11 在 7999 次迭代中的资源申请量为: 0.0
agent 12 在 7999 次迭代中的资源申请量为: 0.0
agent 13 在 7999 次迭代中的资源申请量为: 0.0
agent 14 在 7999 次迭代中的资源申请量为: 0.0
agent 15 在 7999 次迭代中的资源申请量为: 0.0
agent 16 在 7999 次迭代中的资源申请量为: 0.0
agent 17 在 7999 次迭代中的资源申请量为: 0.0
agent 18 在 7999 次迭代中的资源申请量为: 0.0
agent 19 在 7999 次迭代中的资源申请量为: 0.0
agent 20 在 7999 次迭代中的资源申请量为: 0.0
agent 21 在 7999 次迭代中的资源申请量为: 0.0
agent 22 在 7999 次迭代中的资源申请量为: 0.0
agent 23 在 7999 次迭代中的资源申请量为: 0.0
agent 24 在 7999 次迭代中的资源申请量为: 0.0
agent 25 在 7999 次迭代中的资源申请量为: 0.0
agent 26 在 7999 次迭代中的资源申请量为: 0.0
agent 27 在 7999 次迭代中的资源申请量为: 0.0
agent 28 在 7999 次迭代中的资源申请量为: 0.0
agent 29 在 7999 次迭代中的资源申请量为: 0.0
agent 30 在 7999 次迭代中的资源申请量为: 0.0
agent

9999 次迭代中的结果：
agent 1 在 9999 次迭代中的资源申请量为: 0.0
agent 2 在 9999 次迭代中的资源申请量为: 0.0
agent 3 在 9999 次迭代中的资源申请量为: 0.0
agent 4 在 9999 次迭代中的资源申请量为: 0.0
agent 5 在 9999 次迭代中的资源申请量为: 0.0
agent 6 在 9999 次迭代中的资源申请量为: 0.0
agent 7 在 9999 次迭代中的资源申请量为: 0.0
agent 8 在 9999 次迭代中的资源申请量为: 0.0
agent 9 在 9999 次迭代中的资源申请量为: 0.0
agent 10 在 9999 次迭代中的资源申请量为: 0.0
agent 11 在 9999 次迭代中的资源申请量为: 0.0
agent 12 在 9999 次迭代中的资源申请量为: 0.0
agent 13 在 9999 次迭代中的资源申请量为: 0.0
agent 14 在 9999 次迭代中的资源申请量为: 0.0
agent 15 在 9999 次迭代中的资源申请量为: 0.0
agent 16 在 9999 次迭代中的资源申请量为: 0.0
agent 17 在 9999 次迭代中的资源申请量为: 0.0
agent 18 在 9999 次迭代中的资源申请量为: 0.0
agent 19 在 9999 次迭代中的资源申请量为: 0.0
agent 20 在 9999 次迭代中的资源申请量为: 0.0
agent 21 在 9999 次迭代中的资源申请量为: 0.0
agent 22 在 9999 次迭代中的资源申请量为: 0.0
agent 23 在 9999 次迭代中的资源申请量为: 0.0
agent 24 在 9999 次迭代中的资源申请量为: 0.0
agent 25 在 9999 次迭代中的资源申请量为: 0.0
agent 26 在 9999 次迭代中的资源申请量为: 0.0
agent 27 在 9999 次迭代中的资源申请量为: 0.0
agent 28 在 9999 次迭代中的资源申请量为: 0.0
agent 29 在 9999 次迭代中的资源申请量为: 0.0
agent 30 在 9999 次迭代中的资源申请量为: 0.0
agent

...

51999 次迭代中的结果：
agent 1 在 51999 次迭代中的资源申请量为: 0.0
agent 2 在 51999 次迭代中的资源申请量为: 0.0
agent 3 在 51999 次迭代中的资源申请量为: 0.0
agent 4 在 51999 次迭代中的资源申请量为: 0.0
agent 5 在 51999 次迭代中的资源申请量为: 0.0
agent 6 在 51999 次迭代中的资源申请量为: 0.0
agent 7 在 51999 次迭代中的资源申请量为: 0.0
agent 8 在 51999 次迭代中的资源申请量为: 0.0
agent 9 在 51999 次迭代中的资源申请量为: 0.0
agent 10 在 51999 次迭代中的资源申请量为: 0.0
agent 11 在 51999 次迭代中的资源申请量为: 0.0
agent 12 在 51999 次迭代中的资源申请量为: 0.0
agent 13 在 51999 次迭代中的资源申请量为: 0.0
agent 14 在 51999 次迭代中的资源申请量为: 0.0
agent 15 在 51999 次迭代中的资源申请量为: 0.0
agent 16 在 51999 次迭代中的资源申请量为: 0.0
agent 17 在 51999 次迭代中的资源申请量为: 0.0
agent 18 在 51999 次迭代中的资源申请量为: 0.0
agent 19 在 51999 次迭代中的资源申请量为: 0.0
agent 20 在 51999 次迭代中的资源申请量为: 0.0
agent 21 在 51999 次迭代中的资源申请量为: 0.0
agent 22 在 51999 次迭代中的资源申请量为: 0.0
agent 23 在 51999 次迭代中的资源申请量为: 0.0
agent 24 在 51999 次迭代中的资源申请量为: 0.0
agent 25 在 51999 次迭代中的资源申请量为: 0.0
agent 26 在 51999 次迭代中的资源申请量为: 0.0
agent 27 在 51999 次迭代中的资源申请量为: 0.0
agent 28 在 51999 次迭代中的资源申请量为: 0.0
agent 29 在 51999 次迭代中的资源申请量为: 0.0
agent 30

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 0.0
agent 2 在 53999 次迭代中的资源申请量为: 0.0
agent 3 在 53999 次迭代中的资源申请量为: 0.0
agent 4 在 53999 次迭代中的资源申请量为: 0.0
agent 5 在 53999 次迭代中的资源申请量为: 0.0
agent 6 在 53999 次迭代中的资源申请量为: 0.0
agent 7 在 53999 次迭代中的资源申请量为: 0.0
agent 8 在 53999 次迭代中的资源申请量为: 0.0
agent 9 在 53999 次迭代中的资源申请量为: 0.0
agent 10 在 53999 次迭代中的资源申请量为: 0.0
agent 11 在 53999 次迭代中的资源申请量为: 0.0
agent 12 在 53999 次迭代中的资源申请量为: 0.0
agent 13 在 53999 次迭代中的资源申请量为: 0.0
agent 14 在 53999 次迭代中的资源申请量为: 0.0
agent 15 在 53999 次迭代中的资源申请量为: 0.0
agent 16 在 53999 次迭代中的资源申请量为: 0.0
agent 17 在 53999 次迭代中的资源申请量为: 0.0
agent 18 在 53999 次迭代中的资源申请量为: 0.0
agent 19 在 53999 次迭代中的资源申请量为: 0.0
agent 20 在 53999 次迭代中的资源申请量为: 0.0
agent 21 在 53999 次迭代中的资源申请量为: 0.0
agent 22 在 53999 次迭代中的资源申请量为: 0.0
agent 23 在 53999 次迭代中的资源申请量为: 0.0
agent 24 在 53999 次迭代中的资源申请量为: 0.0
agent 25 在 53999 次迭代中的资源申请量为: 0.0
agent 26 在 53999 次迭代中的资源申请量为: 0.0
agent 27 在 53999 次迭代中的资源申请量为: 0.0
agent 28 在 53999 次迭代中的资源申请量为: 0.0
agent 29 在 53999 次迭代中的资源申请量为: 0.0
agent 30

55999 次迭代中的结果：
agent 1 在 55999 次迭代中的资源申请量为: 0.0
agent 2 在 55999 次迭代中的资源申请量为: 0.0
agent 3 在 55999 次迭代中的资源申请量为: 0.0
agent 4 在 55999 次迭代中的资源申请量为: 0.0
agent 5 在 55999 次迭代中的资源申请量为: 0.0
agent 6 在 55999 次迭代中的资源申请量为: 0.0
agent 7 在 55999 次迭代中的资源申请量为: 0.0
agent 8 在 55999 次迭代中的资源申请量为: 0.0
agent 9 在 55999 次迭代中的资源申请量为: 0.0
agent 10 在 55999 次迭代中的资源申请量为: 0.0
agent 11 在 55999 次迭代中的资源申请量为: 0.0
agent 12 在 55999 次迭代中的资源申请量为: 0.0
agent 13 在 55999 次迭代中的资源申请量为: 0.0
agent 14 在 55999 次迭代中的资源申请量为: 0.0
agent 15 在 55999 次迭代中的资源申请量为: 0.0
agent 16 在 55999 次迭代中的资源申请量为: 0.0
agent 17 在 55999 次迭代中的资源申请量为: 0.0
agent 18 在 55999 次迭代中的资源申请量为: 0.0
agent 19 在 55999 次迭代中的资源申请量为: 0.0
agent 20 在 55999 次迭代中的资源申请量为: 0.0
agent 21 在 55999 次迭代中的资源申请量为: 0.0
agent 22 在 55999 次迭代中的资源申请量为: 0.0
agent 23 在 55999 次迭代中的资源申请量为: 0.0
agent 24 在 55999 次迭代中的资源申请量为: 0.0
agent 25 在 55999 次迭代中的资源申请量为: 0.0
agent 26 在 55999 次迭代中的资源申请量为: 0.0
agent 27 在 55999 次迭代中的资源申请量为: 0.0
agent 28 在 55999 次迭代中的资源申请量为: 0.0
agent 29 在 55999 次迭代中的资源申请量为: 0.0
agent 30

57999 次迭代中的结果：
agent 1 在 57999 次迭代中的资源申请量为: 0.0
agent 2 在 57999 次迭代中的资源申请量为: 0.0
agent 3 在 57999 次迭代中的资源申请量为: 0.0
agent 4 在 57999 次迭代中的资源申请量为: 0.0
agent 5 在 57999 次迭代中的资源申请量为: 0.0
agent 6 在 57999 次迭代中的资源申请量为: 0.0
agent 7 在 57999 次迭代中的资源申请量为: 0.0
agent 8 在 57999 次迭代中的资源申请量为: 0.0
agent 9 在 57999 次迭代中的资源申请量为: 0.0
agent 10 在 57999 次迭代中的资源申请量为: 0.0
agent 11 在 57999 次迭代中的资源申请量为: 0.0
agent 12 在 57999 次迭代中的资源申请量为: 0.0
agent 13 在 57999 次迭代中的资源申请量为: 0.0
agent 14 在 57999 次迭代中的资源申请量为: 0.0
agent 15 在 57999 次迭代中的资源申请量为: 0.0
agent 16 在 57999 次迭代中的资源申请量为: 0.0
agent 17 在 57999 次迭代中的资源申请量为: 0.0
agent 18 在 57999 次迭代中的资源申请量为: 0.0
agent 19 在 57999 次迭代中的资源申请量为: 0.0
agent 20 在 57999 次迭代中的资源申请量为: 0.0
agent 21 在 57999 次迭代中的资源申请量为: 0.0
agent 22 在 57999 次迭代中的资源申请量为: 0.0
agent 23 在 57999 次迭代中的资源申请量为: 0.0
agent 24 在 57999 次迭代中的资源申请量为: 0.0
agent 25 在 57999 次迭代中的资源申请量为: 0.0
agent 26 在 57999 次迭代中的资源申请量为: 0.0
agent 27 在 57999 次迭代中的资源申请量为: 0.0
agent 28 在 57999 次迭代中的资源申请量为: 0.0
agent 29 在 57999 次迭代中的资源申请量为: 0.0
agent 30

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 0.0
agent 2 在 59999 次迭代中的资源申请量为: 0.0
agent 3 在 59999 次迭代中的资源申请量为: 0.0
agent 4 在 59999 次迭代中的资源申请量为: 0.0
agent 5 在 59999 次迭代中的资源申请量为: 0.0
agent 6 在 59999 次迭代中的资源申请量为: 0.0
agent 7 在 59999 次迭代中的资源申请量为: 0.0
agent 8 在 59999 次迭代中的资源申请量为: 0.0
agent 9 在 59999 次迭代中的资源申请量为: 0.0
agent 10 在 59999 次迭代中的资源申请量为: 0.0
agent 11 在 59999 次迭代中的资源申请量为: 0.0
agent 12 在 59999 次迭代中的资源申请量为: 0.0
agent 13 在 59999 次迭代中的资源申请量为: 0.0
agent 14 在 59999 次迭代中的资源申请量为: 0.0
agent 15 在 59999 次迭代中的资源申请量为: 0.0
agent 16 在 59999 次迭代中的资源申请量为: 0.0
agent 17 在 59999 次迭代中的资源申请量为: 0.0
agent 18 在 59999 次迭代中的资源申请量为: 0.0
agent 19 在 59999 次迭代中的资源申请量为: 0.0
agent 20 在 59999 次迭代中的资源申请量为: 0.0
agent 21 在 59999 次迭代中的资源申请量为: 0.0
agent 22 在 59999 次迭代中的资源申请量为: 0.0
agent 23 在 59999 次迭代中的资源申请量为: 0.0
agent 24 在 59999 次迭代中的资源申请量为: 0.0
agent 25 在 59999 次迭代中的资源申请量为: 0.0
agent 26 在 59999 次迭代中的资源申请量为: 0.0
agent 27 在 59999 次迭代中的资源申请量为: 0.0
agent 28 在 59999 次迭代中的资源申请量为: 0.0
agent 29 在 59999 次迭代中的资源申请量为: 0.0
agent 30

In [6]:
np.savetxt('./data/00_DRA_agent_resource_quantity_max_attack.txt',agent_resource_quantity_attack2,fmt='%0.8f')#用户资源
np.savetxt('./data/00_DRA_average_cost_honest_max_attack.txt',average_cost_honest_attack2,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/00_DRA_average_resource_honest_max_attack.txt',average_resource_honest_attack2,fmt='%0.8f')#诚实用户平均资源
np.savetxt('./data/00_DRA_dual_variable_max_attack.txt',dual_variable_attack2,fmt='%0.8f')#用户对偶变量